# Links and Sources

### Energy Information Administration (EIA)
The U.S. Energy Information Administration (EIA) is a principal agency of the U.S. Federal Statistical System responsible for collecting, analyzing, and disseminating energy information. <br>
https://www.eia.gov/


### Electric Sales, Revenue, and Average Price
https://www.eia.gov/electricity/sales_revenue_price/index.php

### Retail Sales - All (Residential, Commercial, Industrial, Transportation)
https://www.eia.gov/electricity/sales_revenue_price/pdf/table_10.pdf <br>
Gives : City, State, #Customers, # MWhrs, # dollars, Average energy price

### Terminology

Difference between Privately owned utilities, Cooperatives, Municipalities : https://ceateam.com/whats-difference-privately-owned-utilities-cooperatives-municipalities/

# Plan 

1. Get demand data from for all the possible cities in US
2. Get demand data from all other sources (excluding cities) for each state
3. Get the average demand per sq.miles for each state using data from (2)
3. Find a way to see what cities are within a given distance 'd' from any given coordinates
4. The total demand at any given coordinate will be the sum of 
    1. The demand from all the cities included in distance 'd' around it
    2. **The total demand from all other sources calculated using (2)**

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import ast

In [78]:
def coordinates_to_state(lat, lon):
    # Using google maps API
    return state_name

def electricity_demand_within_distance(distance):
    # distance - miles
    # electricity demand - MW
    
    return demand

In [32]:
data = pd.read_excel('dataset_energy_demand.xlsx')
data.columns = data.iloc[1]
data.drop(data.index[0:2], inplace=True)
data.reset_index(drop=True,inplace=True)
data

1,Entity,State,Ownership,Customers (Count),Sales (Megawatthours),Revenues (Thousands Dollars),Average Price (cents/kWh)
0,Alaska Electric Light & Power Co.,AK,Investor Owned,17701,405072,46496.8,11.478651
1,Alaska Power and Telephone Co,AK,Investor Owned,8289,73691,23750,32.229173
2,"Alaska Village Elec Coop, Inc",AK,Cooperative,11512,124689,60171.8,48.257505
3,Chugach Electric Assn Inc,AK,Cooperative,113096,1902625,322098.3,16.929153
4,Golden Valley Elec Assn Inc,AK,Cooperative,47669,1244414,282561.3,22.706374
...,...,...,...,...,...,...,...
1642,Adjustment 2022,VT,Other,33230,442257,71575.1,.
1643,Adjustment 2022,WA,Other,67197,1396093,133891.3,.
1644,Adjustment 2022,WI,Other,184746,3367431,418071.9,.
1645,Adjustment 2022,WV,Other,11160,155580,22923.4,.


In [3]:
data['Entity'][data['Ownership']=='Municipal']

6                  Ketchikan Public Utilities
12      Albertville Municipal Utilities Board
18                          City of Andalusia
19                      City of Athens - (AL)
20                 City of Bessemer Utilities
                        ...                  
1579              Waterloo Light & Water Comm
1580                         Waupun Utilities
1582               Whitehall Electric Utility
1586            Wisconsin Rapids W W & L Comm
1594                  City of Gillette - (WY)
Name: Entity, Length: 455, dtype: object

In [4]:
data['Ownership'].unique()

array(['Investor Owned', 'Cooperative', 'Municipal', 'Federal', nan,
       'State', 'Political Subdivision', 'Retail Energy Provider',
       'Other'], dtype=object)

In [8]:
data['State'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [13]:

len(data['Entity'].unique())#.head(n)

1308

In [33]:
np.set_printoptions(threshold=np.inf)

n = 1308
count = 0
a = data['Entity'].unique()
for x in a:
    if 'City of' in x:
        print(x)
        count+=1
print(count)

City of Andalusia
City of Athens - (AL)
City of Bessemer Utilities
City of Courtland
City of Dothan - (AL)
City of Florence - (AL)
City of Hartselle
City of Huntsville - (AL)
City of Muscle Shoals
City of Opelika - (AL)
City of Russellville - (AL)
City of Scottsboro
City of Tarrant
City of Troy - (AL)
City of Tuscumbia
City of Benton - (AR)
City of Bentonville - (AR)
City of Hope
City of North Little Rock - (AR)
City of Siloam Springs - (AR)
City of West Memphis - (AR)
City of Mesa - (AZ)
City of Anaheim - (CA)
City of Azusa
City of Burbank Water and Power
City of Colton - (CA)
City of Corona - (CA)
City of Glendale - (CA)
City of Lodi - (CA)
City of Moreno Valley - (CA)
City of Palo Alto - (CA)
City of Pasadena - (CA)
City of Redding - (CA)
City of Riverside - (CA)
City of Roseville - (CA)
City of Santa Clara - (CA)
City of Vernon
City of Colorado Springs - (CO)
City of Fort Collins - (CO)
City of Fort Morgan
City of Fountain
City of Longmont
City of Loveland - (CO)
City of Norwich - 

In [44]:
import requests

city = 'Lodi'
country = 'USA'
api_url = 'https://api.api-ninjas.com/v1/geocoding?city={}&country={}'.format(city,country)
response = requests.get(api_url + city, headers={'X-Api-Key': 'MUFmQYBgNJyE/VmlfN+wNA==aPmARELVzoFWzbT0'})
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)

[{"name": "Lodi", "latitude": 45.3138959, "longitude": 9.503263, "country": "IT", "state": "Lombardy"}, {"name": "Lodi", "latitude": 38.1301968, "longitude": -121.2724473, "country": "US", "state": "California"}, {"name": "Lodi", "latitude": 38.092239500000005, "longitude": -121.38984593366523, "country": "US", "state": "California"}, {"name": "Lodi", "latitude": 40.8823215, "longitude": -74.0831971, "country": "US", "state": "New Jersey"}, {"name": "Town of Lodi", "latitude": 42.6139597, "longitude": -76.8224557, "country": "US", "state": "New York"}]


In [81]:
res = json.loads(response.text)
a = ast.literal_eval(str(res[2]))
a

{'name': 'Lodi',
 'latitude': 38.092239500000005,
 'longitude': -121.38984593366523,
 'country': 'US',
 'state': 'California'}